In [144]:
# Import required libraries
import pandas as pd
import re
from datetime import datetime

In [145]:
# Get sheets from excel files and store in pandas dataframes
xlsx = pd.ExcelFile('Week7Challenge.xlsx')
alloc_details_df = pd.read_excel(xlsx, 'Allocation Details')
dep_details_df = pd.read_excel(xlsx, 'Departure Details')

In [146]:
# Explore the data
#alloc_details_df.head()
#dep_details_df.head()
#alloc_details_df_temp.head()

In [147]:
#Generate Ship ID fields from Departure ID
alloc_details_df['Ship ID'] = alloc_details_df['Departure ID'].str.rsplit('-',3,expand=True)[0]

In [148]:
#Generate Departure Date fields from Departure ID
alloc_details_df['Departure Date'] = alloc_details_df['Departure ID'].str.split('-',2,expand=True)[2]

In [149]:
#Convertt Departure Date to datetime field
alloc_details_df['Departure Date'] = pd.to_datetime(alloc_details_df['Departure Date'],format='%d-%m-%Y')

In [150]:
# Group by Ship ID & Departure Date and sum on Weight & Volume Allocated fields
aggregations = {
    'Weight Allocated':'sum',
    'Volume Allocated': 'sum'
}
alloc_details_df_temp = alloc_details_df.groupby(
    ['Ship ID','Departure Date'], as_index=False
    ).agg(aggregations)

In [151]:
alloc_details_df_temp.head()

,Ship ID,Departure Date,Weight Allocated,Volume Allocated
0,Freighter-01,2019-04-06,1336,112
1,Freighter-01,2019-06-18,4580,556
2,Freighter-02,2019-04-10,3972,396
3,Freighter-02,2019-04-19,1932,200
4,Freighter-02,2019-05-26,4252,294


In [152]:
# Join alloc_details_df_temp with dep_details_df 
# on Ship ID and Departure Date
dep_details_df_final = pd.merge(dep_details_df, 
                                alloc_details_df_temp, 
                                on=['Ship ID','Departure Date'], 
                                how='inner')

In [153]:
# Check if data is ok
#dep_details_df_final[dep_details_df_final['Ship ID']=='Tug-05']

In [154]:
# Define 2 functions to check if Allocated Weight/Volume > Max Weight/Volume
def f(row):
    if row['Weight Allocated'] > row['Max Weight']:
        return True
    else:
        return False
def g(row):
    if row['Volume Allocated'] > row['Max Volume']:
        return True
    else:
        return False

In [155]:
# Create Max Weight Exceeded? & Max Vol Exceeded? fields using apply()
dep_details_df_final['Max Weight Exceeded?'] = dep_details_df_final.apply(f, axis=1)
dep_details_df_final['Max Vol Exceeded?'] = dep_details_df_final.apply(g, axis=1)

In [156]:
# Check Data
#dep_details_df_final[dep_details_df_final['Ship ID']=='Tug-05']
dep_details_df_final[dep_details_df_final['Ship ID']=='Freighter-01']

,Ship ID,Departure Date,Max Weight,Max Volume,Weight Allocated,Volume Allocated,Max Weight Exceeded?,Max Vol Exceeded?
3,Freighter-01,2019-04-06,5000,500,1336,112,False,False
52,Freighter-01,2019-06-18,5000,500,4580,556,False,True


In [157]:
dep_details_df_final

,Ship ID,Departure Date,Max Weight,Max Volume,Weight Allocated,Volume Allocated,Max Weight Exceeded?,Max Vol Exceeded?
0,Tug-05,2019-03-27,500,50,707,68,True,True
1,Freighter-04,2019-03-31,5000,500,616,106,False,False
2,Freighter-03,2019-04-01,5000,500,2392,232,False,False
3,Freighter-01,2019-04-06,5000,500,1336,112,False,False
4,Tanker-02,2019-04-07,1000,200,809,110,False,False
5,Tanker-04,2019-04-10,1000,200,257,58,False,False
6,Freighter-02,2019-04-10,5000,500,3972,396,False,False
7,Tug-05,2019-04-10,500,50,222,27,False,False
8,Freighter-03,2019-04-10,5000,500,3476,456,False,False
9,Freighter-04,2019-04-12,5000,500,2120,176,False,False
